In [1]:
print(1)
from qiskit.quantum_info import Statevector, Pauli
from qiskit.circuit.library import HamiltonianGate, HGate
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathos.multiprocessing import ProcessingPool
from functools import partial
import multiprocessing

1


In [2]:
print(2)
HO=1/np.sqrt(2)*np.array([[1.0, 1.0],[1.0,-1.0]])
UO=1/np.sqrt(2)*np.array([[1.0, 0-1.0j],[1,0+1.0j]])

2


In [3]:
print(3)
def init(N):
    stavec = Statevector.from_int(0,2**(N+1))
    for i in range(N+1):
        stavec= stavec.evolve(HO,[i])
    return stavec


def measure_reset(sv):
    sv = sv.evolve(UO,[0])
    meas,sv = sv.measure([0])
    sv=sv.reset([0])
    sv=sv.evolve(HO,[0])
    return sv

3


In [4]:
print(4)
def evol_ZZ(sv,U1):
    N = sv.num_qubits-1
    for i in range(N):
        sv=sv.evolve(U1,[0,i+1,(i+1)% N + 1])
        sv = measure_reset(sv)
    sv = measure_reset(sv)
    return sv

def evol_X(sv,U2):
    N = sv.num_qubits-1
    for i in range(1,N+1):
        sv=sv.evolve(U2,[i,0])
        sv = measure_reset(sv)
    return sv 

4


In [5]:
print(5)
def Get_ExpectValue_ZZ(sv,N):
    EV_list=[]
    for i in range(int(N/2)):
        EV=[]
        for j in range(N):
            EV.append(np.real(sv.expectation_value(Pauli('ZZ'),[j+1,(j+1+i)%N+1])) ** 2)
        EV_list.append(np.average(EV))
    return EV_list

5


In [6]:
print(6)
def Run_ZZX(Num_qubits,ZZ_time,X_time,Sample,Depth):
    N = Num_qubits
    t1 = ZZ_time
    t2 = X_time
    U1 = HamiltonianGate(Pauli('ZZZ'),t1)
    U2 = HamiltonianGate(Pauli('ZX'),t2)
    EV_Sams=[]
    for i in range(Sample):
        sv=init(N)
        EV_Sam=[]
        for j in range(Depth):
            sv=evol_ZZ(sv,U1)
            sv=evol_X(sv,U2)
            EV_Sam.append(Get_ExpectValue_ZZ(sv,N))
        EV_Sams.append(EV_Sam)
    EV_Ave = np.sum(EV_Sams,axis=0) / Sample
#    print(EV_Ave)
    name = "EV_Data/"+"ZZX_EV"+"+"+str(N)+ "+" + str(t1) + "+" + str(t2) + ".npy" 
    np.save(name,EV_Ave)

6


In [7]:
print(7)
a=[4,6,8,10,12]
b=[0.02,0.1,0.18]

Num_qubits=[]
ZZ_time=[]
X_time=[]

for i in range(5):
    for j in range(3):
        for k in range(3):
            Num_qubits.append(a[i])
            ZZ_time.append(b[j])
            X_time.append(b[k])

7


In [ ]:
print(8)
if __name__=='__main__':
    Partial_Run=partial(Run_ZZX,Sample=1000,Depth=30)
    pool = ProcessingPool(multiprocessing.cpu_count())
    pool.map(Partial_Run, Num_qubits, ZZ_time, X_time)
    pool.join()
    pool.close()

8
[[1.89525172e-03 3.70891653e-06]
 [2.68109937e-03 1.48758045e-05]
 [5.99912939e-03 7.47197315e-05]][[1.70209351e-03 6.59541354e-06]
 [2.19281229e-03 1.06960544e-05]
 [6.58991065e-03 4.88199301e-05]]

[[1.55712021e-03 4.00915617e-06]
 [3.31546476e-03 1.65505840e-05]
 [4.87749219e-03 3.72302288e-05]]
[[0.030453   0.00157575]
 [0.04658648 0.00515518]
 [0.07746797 0.01070366]]
[[0.04252319 0.00592199]
 [0.10884652 0.02948667]
 [0.18219653 0.08077211]]
[[0.10925968 0.01833549]
 [0.25150577 0.13695864]
 [0.40452111 0.30247806]]
[[0.12379841 0.02462626]
 [0.20519001 0.06061822]
 [0.33463785 0.17081615]]
[[0.15230398 0.04315971]
 [0.23292619 0.07781993]
 [0.32389503 0.15755981]]
[[0.0380082  0.00189636]
 [0.05337492 0.00630995]
 [0.08891273 0.01324627]]
[[1.80547499e-03 3.23040177e-06 1.43313923e-08]
 [4.09275807e-03 1.28234490e-05 1.18428222e-07]
 [3.90915450e-03 1.29810359e-05 1.06557610e-07]]
[[1.61723055e-03 2.60191243e-06 6.79857565e-09]
 [3.02459635e-03 9.90425574e-06 7.19123249e-08]
 

KeyboardInterrupt: 

[[1.25241103e-01 1.56545946e-02 1.95728015e-03 2.43984117e-04
  3.01389519e-05 6.68273935e-06]
 [2.05938772e-01 4.91487567e-02 1.04700394e-02 1.83410084e-03
  2.26821695e-04 6.24782093e-05]
 [2.74065752e-01 8.70204909e-02 2.35762365e-02 6.82324870e-03
  2.36744502e-03 1.22993971e-03]]
